## Load/import packages

In [4]:
import json
import cv2
import scipy
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential, layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from os import listdir, mkdir
from os.path import splitext
from os.path import join
from skimage import io, color

%matplotlib inline

# Check if Tensorflow uses GPU
print(tf.config.experimental.list_physical_devices("GPU"))

# Limit GPU memory usage
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

print()
print(f"Tensorflow Version: {tf.__version__}")
print(f"Numpy Version: {np.__version__}")
# print(f"OpenCV Version: {cv2.__version__}")
print(f"Matplotlib Version: {matplotlib.__version__}")
print(f"Keras Version: {tf.keras.__version__}")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Tensorflow Version: 2.4.0-rc0
Numpy Version: 1.19.2
Matplotlib Version: 3.3.1
Keras Version: 2.4.0


# Load Features

In [6]:
train_features = scipy.sparse.load_npz('data/train_features.npz')
val_features = scipy.sparse.load_npz("data/val_features.npz")
train_labels = np.load("data/train_labels.npy")
val_labels = np.load("data/val_labels.npy")

1. Load the .npz and .npy files for training and validation set
2. The features are starting from Class 0 till Class 7 in Ascending order
3. For each class, take all features of each video, take this as sequence
    3.1 Look up what the largest sequence length is
    3.2 Pad all other sequences to the largest sequence length
4. Reshape the features to [sequences, sequence_length, features]


In [13]:
# Setup up model with base_VGG19 and all those layers frozen
# Define the sequence input size

def build_model(inp_shape):
    model = Sequential(name="RNN-LSTM")
    model.add(layers.LSTM(16, input_shape=inp_shape, return_sequences=True, dropout=0.5, name="LSTM_1"))
    model.add(layers.Dense(7, activation="softmax", name="Dense_Output"))
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])
    return model

In [12]:
cnn_rnn_LSTM = build_model((1,1))
cnn_rnn_LSTM.summary()

Model: "RNN-LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM_1 (LSTM)                (None, 1, 16)             1152      
_________________________________________________________________
Dense_Output (Dense)         (None, 1, 7)              119       
Total params: 1,271
Trainable params: 1,271
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# AW2_norm_minitrain = AW2_norm_minitrain.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

history = cnn_rnn_LSTM.fit(
    AW2_norm_train.batch(32, drop_remainder=True),
    validation_data=AW2_norm_val.batch(32, drop_remainder=True),
    epochs=10,
    verbose=1,
)

In [ ]:
# cnn_rnn_LSTM.evaluate(AW2_norm_train.batch(8, drop_remainder=True))